## Install master branch of TensorTrade

In [0]:
!pip install git+https://github.com/tensortrade-org/tensortrade.git -U

In [3]:
import tensortrade.env.default as default

from tensortrade.feed.core import Stream, DataFeed
from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.oms.wallets import Portfolio, Wallet
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH, LTC

## Load Data for Exchanges

Using the `tensortrade.data.cdd` module you can load data from any `csv` file provided at:
- https://www.cryptodatadownload.com/data/northamerican/

using the `CryptoDataDownload` class.

In [4]:
cdd = CryptoDataDownload()
coinbase_btc = cdd.fetch("Coinbase", "USD", "BTC", "1h")
coinbase_eth = cdd.fetch("Coinbase", "USD", "ETH", "1h")

bitstamp_btc = cdd.fetch("Bitstamp", "USD", "BTC", "1h")
bitstamp_eth = cdd.fetch("Bitstamp", "USD", "ETH", "1h")
bitstamp_ltc = cdd.fetch("Bitstamp", "USD", "LTC", "1h")

## Inspect Transactions of `SimpleOrders`

In [5]:
coinbase = Exchange("coinbase", service=execute_order)(
    Stream.source(list(coinbase_btc['close'][-100:]), dtype="float").rename("USD-BTC"),
    Stream.source(list(coinbase_eth['close'][-100:]), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_btc['close'][-100:]), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_eth['close'][-100:]), dtype="float").rename("USD-ETH"),
    Stream.source(list(bitstamp_ltc['close'][-100:]), dtype="float").rename("USD-LTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
    Wallet(coinbase, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 20 * ETH),
    Wallet(bitstamp, 3 * LTC)
])

feed = DataFeed([
    Stream.source(list(bitstamp_eth['volume'][-100:]), dtype="float").rename("volume:/USD-ETH"),
    Stream.source(list(bitstamp_ltc['volume'][-100:]), dtype="float").rename("volume:/USD-LTC")
])

env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.SimpleOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)

done = False
obs = env.reset()
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

portfolio.ledger.as_frame().head(7)

poid  step               source  \
0  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1    bitstamp:USD/free   
1  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
2  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
3  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1             bitstamp   
4  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
5  ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:BTC/locked   
6  9ba20790-6b44-4f65-8cc3-545528dda301     2    coinbase:BTC/free   

                target                                               memo  \
0  bitstamp:USD/locked                              LOCK (LOCK FOR ORDER)   
1             bitstamp                            WITHDRAWAL (COMMISSION)   
2             bitstamp                            WITHDRAWAL (FILL ORDER)   
3  bitstamp:BTC/locked  DEPOSIT (TRADED 997.00 USD bitstamp:USD/BTC @ ...   
4    bitstamp:USD/free                             UNLOCK USD (COMPLETED)   
5    bitstamp:BTC/free                             UNLOCK BTC (COMPLETED)   
6  coinbase:BTC/locked                              LOCK (LOCK FOR ORDER)   

           amount            free          locked     locked_poid  
0     1000.00 USD        0.00 USD     1000.00 USD     1000.00 USD  
1        3.00 USD        0.00 USD      997.00 USD      997.00 USD  
2      997.00 USD        0.00 USD        0.00 USD        0.00 USD  
3  0.10940979 BTC  5.00000000 BTC  0.10940979 BTC  0.10940979 BTC  
4        0.00 USD        0.00 USD        0.00 USD        0.00 USD  
5  0.10940979 BTC  5.10940979 BTC  0.00000000 BTC  0.00000000 BTC  
6  3.00000000 BTC  7.00000000 BTC  3.00000000 BTC  3.00000000 BTC

## Inspect Transactions of `ManagedRiskOrders`

In [6]:
portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 0 * BTC),
    Wallet(coinbase, 0 * ETH),
])

env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.ManagedRiskOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)

done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

portfolio.ledger.as_frame().head(20)

poid  step               source  \
0   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1    bitstamp:USD/free   
1   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
2   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
3   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1             bitstamp   
4   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:USD/locked   
5   ba218138-98c9-4e2c-b2fd-c12c79f2a07e     1  bitstamp:BTC/locked   
6   9ba20790-6b44-4f65-8cc3-545528dda301     2    coinbase:BTC/free   
7   9ba20790-6b44-4f65-8cc3-545528dda301     2  coinbase:BTC/locked   
8   9ba20790-6b44-4f65-8cc3-545528dda301     2  coinbase:BTC/locked   
9   9ba20790-6b44-4f65-8cc3-545528dda301     2             coinbase   
10  9ba20790-6b44-4f65-8cc3-545528dda301     2  coinbase:USD/locked   
11  9ba20790-6b44-4f65-8cc3-545528dda301     2  coinbase:BTC/locked   
12  bc01cc16-082f-44ab-8ec9-9599360caa4d     3    coinbase:ETH/free   
13  bc01cc16-082f-44ab-8ec9-9599360caa4d     3  coinbase:ETH/locked   
14  bc01cc16-082f-44ab-8ec9-9599360caa4d     3  coinbase:ETH/locked   
15  bc01cc16-082f-44ab-8ec9-9599360caa4d     3             coinbase   
16  bc01cc16-082f-44ab-8ec9-9599360caa4d     3  coinbase:USD/locked   
17  bc01cc16-082f-44ab-8ec9-9599360caa4d     3  coinbase:ETH/locked   
18  2f91580e-6da9-4e4d-9fc1-cc26eed1a65f     4    coinbase:BTC/free   
19  2f91580e-6da9-4e4d-9fc1-cc26eed1a65f     4  coinbase:BTC/locked   

                 target                                               memo  \
0   bitstamp:USD/locked                              LOCK (LOCK FOR ORDER)   
1              bitstamp                            WITHDRAWAL (COMMISSION)   
2              bitstamp                            WITHDRAWAL (FILL ORDER)   
3   bitstamp:BTC/locked  DEPOSIT (TRADED 997.00 USD bitstamp:USD/BTC @ ...   
4     bitstamp:USD/free                             UNLOCK USD (COMPLETED)   
5     bitstamp:BTC/free                             UNLOCK BTC (COMPLETED)   
6   coinbase:BTC/locked                              LOCK (LOCK FOR ORDER)   
7              coinbase                            WITHDRAWAL (COMMISSION)   
8              coinbase                            WITHDRAWAL (FILL ORDER)   
9   coinbase:USD/locked  DEPOSIT (TRADED 2.99100000 BTC coinbase:USD/BT...   
10    coinbase:USD/free                             UNLOCK USD (COMPLETED)   
11    coinbase:BTC/free                             UNLOCK BTC (COMPLETED)   
12  coinbase:ETH/locked                              LOCK (LOCK FOR ORDER)   
13             coinbase                            WITHDRAWAL (COMMISSION)   
14             coinbase                            WITHDRAWAL (FILL ORDER)   
15  coinbase:USD/locked  DEPOSIT (TRADED 3.98800000 ETH coinbase:USD/ET...   
16    coinbase:USD/free                             UNLOCK USD (COMPLETED)   
17    coinbase:ETH/free                             UNLOCK ETH (COMPLETED)   
18  coinbase:BTC/locked                              LOCK (LOCK FOR ORDER)   
19             coinbase                            WITHDRAWAL (COMMISSION)   

            amount            free          locked     locked_poid  
0      1000.00 USD        0.00 USD     1000.00 USD     1000.00 USD  
1         3.00 USD        0.00 USD      997.00 USD      997.00 USD  
2       997.00 USD        0.00 USD        0.00 USD        0.00 USD  
3   0.10940979 BTC  5.00000000 BTC  0.10940979 BTC  0.10940979 BTC  
4         0.00 USD        0.00 USD        0.00 USD        0.00 USD  
5   0.10940979 BTC  5.10940979 BTC  0.00000000 BTC  0.00000000 BTC  
6   3.00000000 BTC  7.00000000 BTC  3.00000000 BTC  3.00000000 BTC  
7   0.00900000 BTC  7.00000000 BTC  2.99100000 BTC  2.99100000 BTC  
8   2.99100000 BTC  7.00000000 BTC  0.00000000 BTC  0.00000000 BTC  
9     27249.98 USD    10000.00 USD    27249.98 USD    27249.98 USD  
10    27249.98 USD    37249.98 USD        0.00 USD        0.00 USD  
11  0.00000000 BTC  7.00000000 BTC  0.00000000 BTC  0.00000000 BTC  
12  4.00

## Transactions in Spreadsheets

To take a closer look at the transactions that are happening within the system, copy the transactions to a csv file and load it into any spreadsheet software. If where you are running this allows access to the system clipboard, you can directly copy the frame to your system clipboard by doing the following:
- `portfolio.ledger.as_frame().to_clipboard(index=False)`

Then just paste into any spreadsheet software (e.g. Execel, Google Sheets)

image.png